In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from astropy import units as u
from astropy.table import Table 
from astropy.nddata import CCDData
from astropy.coordinates import SkyCoord
import astropy.coordinates as apycoord
import glowing_waffles as gw
import notebook_functions as nf

In [ ]:
ccd = CCDData.read('kelt-16-S001-R001-C100-r.fit')

In [ ]:
our_mags = Table.read('all_the_photometry.fits')

our_mags = our_mags[our_mags['file'] == 'kelt-16-S001-R001-C100-r.fit']

In [ ]:
apass, apass_x, apass_y, apass_for_transform, in_apass_x, in_apass_y = nf.find_apass_stars(ccd)
vsx, vsx_x, vsx_y, vsx_names = nf.find_known_variables(ccd)

apass_all_coords = SkyCoord(apass['RAJ2000'], apass['DEJ2000'], unit='deg')
apass_for_transform_coords = SkyCoord(apass_for_transform['RAJ2000'], apass_for_transform['DEJ2000'], unit='deg')

In [ ]:
our_coords = SkyCoord(our_mags['RA'], our_mags['Dec'])


In [ ]:
apass_for_transform_index, d2d, d3d = apycoord.match_coordinates_sky(our_coords, apass_for_transform_coords)

#create a boolean of all of the matches that have a discrepancy of less then 5 arcseconds
good_match_for_transform = d2d < 5 * u.arcsecond

In [ ]:
apass_index, d2d_a_idx, d3d_a_idx = apycoord.match_coordinates_sky(our_coords, apass_all_coords)

good_match_all = d2d_a_idx < 5 * u.arcsecond
apass_all_indexes = apass_index[good_match_all]

In [ ]:
plt.figure(figsize=(20, 10))

plt.plot(our_mags['mag_inst_r'][good_match_all], apass['r_mag'][apass_all_indexes], '.')
plt.xlabel('instrumental r magnitude')
plt.ylabel('catalog r magnitude')

In [ ]:
our_match_mags = our_mags['mag_inst_r'][good_match_for_transform]

apass_match_indexes = apass_for_transform_index[good_match_for_transform]

apass_match_mags = apass_for_transform['r_mag'][apass_match_indexes]
apass_match_color = apass_for_transform['B-V'][apass_match_indexes]

good_mags = ~np.isnan(our_match_mags)

our_match_mags = our_match_mags[good_mags]
apass_match_mags = apass_match_mags[good_mags]
apass_match_color = apass_match_color[good_mags]

matched_data, transforms = gw.differential_photometry.calculate_transform_coefficients(our_match_mags,
                                                            apass_match_mags,
                                                            apass_match_color, sigma=2
                                                           )

In [ ]:
matched_data2, transforms2 = gw.differential_photometry.calculate_transform_coefficients(our_match_mags,
                                                            apass_match_mags,
                                                            apass_match_color, sigma=2, faintest_mag=14
                                                           )

In [ ]:
plt.figure(figsize=(20, 10))

used_in_fit = ~matched_data.mask
plt.plot(apass_match_color, apass_match_mags - our_match_mags, 'o')
#plt.plot(apass_match_color, matched_data, '+')
plt.plot(apass_match_color, transforms(apass_match_color), 'gx')
plt.plot(apass_match_color, transforms2(apass_match_color), 'gs')
plt.plot(apass_match_color, matched_data2, 'r+')

plt.xlabel('Color (B-V, catalog)')
plt.ylabel('Catalog - instrumental magnitude')

In [ ]:
our_cat_mags = our_mags['mag_inst_r'][good_match_all] + apass['B-V'][apass_all_indexes] * transforms.parameters[1] + transforms.parameters[0]
our_cat_mags2 = our_mags['mag_inst_r'][good_match_all] + apass['B-V'][apass_all_indexes] * transforms2.parameters[1] + transforms2.parameters[0]

In [ ]:
plt.figure(figsize=(20, 10))

plt.plot(our_mags['mag_inst_r'][good_match_all] + transforms2.parameters[0], 
         apass['r_mag'][apass_all_indexes], '.', 
         color='red', alpha=0.8, label='No color correction')
#plt.plot(our_cat_mags, apass['r_mag'][apass_all_indexes], '.', alpha=0.4, label='Color corrected')
plt.plot(our_cat_mags2, apass['r_mag'][apass_all_indexes], '.', alpha=0.4, label='Color corrected, BRIGHT')
plt.plot([10, 16], [10, 16], label='Slope = 1')
plt.xlabel('our catalog r magnitude')
plt.ylabel('catalog r magnitude')
plt.legend()

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(apass['r_mag'][apass_all_indexes], our_cat_mags - apass['r_mag'][apass_all_indexes], '.', color='blue', label='With color corrections')
plt.plot(apass['r_mag'][apass_all_indexes], our_cat_mags2 - apass['r_mag'][apass_all_indexes], '.', color='orange', label='With color corrections')
plt.plot(apass['r_mag'][apass_all_indexes], our_mags['mag_inst_r'][good_match_all] + transforms.parameters[0] - apass['r_mag'][apass_all_indexes], '.', color='green', label='Without color corrections')
plt.grid()
plt.legend()
plt.xlabel('Catalog magnitude')
plt.ylabel('Our magnitude - catalog')

In [ ]:
(our_cat_mags - apass['r_mag'][apass_all_indexes]).std(), (our_cat_mags2 - apass['r_mag'][apass_all_indexes]).std(), (our_mags['mag_inst_r'][good_match_all] + transforms.parameters[0] - apass['r_mag'][apass_all_indexes]).std()

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(apass['B-V'][apass_all_indexes], our_cat_mags - apass['r_mag'][apass_all_indexes], '.', color='blue', label='With color corrections')
plt.plot(apass['B-V'][apass_all_indexes], our_mags['mag_inst_r'][good_match_all] + transforms.parameters[0] - apass['r_mag'][apass_all_indexes], '.', color='green', label='Without color corrections')
plt.plot(apass['B-V'][apass_all_indexes], our_cat_mags2 - apass['r_mag'][apass_all_indexes], '.', color='orange', label='With color corrections BETTER')
plt.grid()
plt.legend()